In [ ]:
import requests
from collections import defaultdict
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='my_app')
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib import cm
import networkx as nx
import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely.geometry.polygon import Point, Polygon
from shapely.geometry import shape
from shapely.ops import unary_union
from shapely.geometry.multipolygon import MultiPolygon
import folium
import geopandas as gpd
import math
from geopy.distance import distance

In [ ]:
lsoa_data = pd.read_csv('../data/lsoa_global_number_residents_2021.csv')

In [ ]:
lsoa_postcode = pd.read_csv('../data/pcd_lsoa21cd_nov22_en.csv')
lsoa_postcode.head(2)

In [ ]:
lsoa_pop = pd.read_csv('../data/lsoa_global_number_residents_2021.csv')
lsoa_pop.head(3)

In [ ]:

G = ox.graph_from_place('Cambridgeshire', network_type='drive')


In [ ]:


gdf = gpd.read_file('../data/LSOA_Dec_2021_Boundaries_Generalised_Clipped_EW_BGC_2022_5000101660793162025/LSOA_2021_EW_BGC.shp')
gdf_cambs = gdf.query("LSOA21NM.str.contains('Cambridge')")


In [ ]:
gdf_c= gdf.query("LSOA21NM.str.contains('Cambridge')")

In [ ]:
len(gdf_c)

In [ ]:
# set the CRS of the GeoDataFrame to British National Grid (EPSG:27700)
gdf_cambs = gdf_cambs.set_crs(epsg=27700)

# project the geometry to WGS84 (EPSG:4326)
gdf_cambs = gdf_cambs.to_crs(epsg=4326)


lsoa_codes = gdf_cambs['LSOA21CD'].tolist()
lsoa_data = {lsoa_code: {} for lsoa_code in lsoa_codes}


In [ ]:
gdf_cambs

In [ ]:

for index, row in gdf_cambs.iterrows():
    lsoa_code = row['LSOA21CD']
    if lsoa_code in lsoa_data:
        lsoa_data[lsoa_code]['Latitude'] = row['geometry'].centroid.y
        lsoa_data[lsoa_code]['Longitude'] = row['geometry'].centroid.x
        node = ox.distance.nearest_nodes(G, row['geometry'].centroid.x, row['geometry'].centroid.y)
        lsoa_data[lsoa_code]['Node'] = node


In [ ]:
lsoa_population = {}
for lsoa_code in lsoa_data:
    population = lsoa_pop.loc[lsoa_pop['LSOA21CD'] == lsoa_code, 'Population'].iloc[0]
    try:
        latitude = lsoa_data[lsoa_code]['Latitude']
    except KeyError:
        print(f'KeyError: Latitude not found for LSOA code {lsoa_code}')
        latitude = None
    try:
        longitude = lsoa_data[lsoa_code]['Longitude']
    except KeyError:
        print(f'KeyError: Longitude not found for LSOA code {lsoa_code}')
        longitude = None
    lsoa_population[lsoa_code] = {'Population': population, 'Latitude': latitude, 'Longitude': longitude}




In [ ]:
lsoa_population

In [ ]:
address= '184 Victoria Road, Cambridge, UK'
radius = 5000
lsoa_data=lsoa_data
G=G
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)
#get_average_travel_times('184 Victoria Road, Cambridge, UK',5000,lsoa_data,G)


In [ ]:
from geopy.geocoders import Nominatim
import networkx as nx
import osmnx as ox
from haversine import haversine, Unit

geolocator = Nominatim(user_agent='my_application')
location = Nominatim(user_agent='my_app').geocode(address)
origin = (location.latitude, location.longitude)

def haversine_distance(origin, destination, speed=40):


    distance_km = haversine(origin, destination, unit='km')
    time_sec = (distance_km / speed) * 3600
    return time_sec

# get LSOAs within search radius
lsoas_in_radius = []
for lsoa_code, data in lsoa_data.items():
    if haversine_distance(origin, (data['Latitude'], data['Longitude'])) <= radius:
        lsoas_in_radius.append(lsoa_code)

# create a list of LSOA codes within search radius
lsoa_codes_in_radius = [lsoa for lsoa in lsoa_population.keys() if lsoa in lsoas_in_radius]





In [ ]:
from geopy.distance import great_circle

def get_average_travel_times(address, radius, lsoa_population, G):


    geolocator = Nominatim(user_agent='my_application')
    location = Nominatim(user_agent='my_app').geocode(address)
    origin = (location.latitude, location.longitude)


    lsoas_in_radius = []

    for lsoa_code, data in lsoa_population.items():
        lsoa_coordinates = (data['Latitude'], data['Longitude'])
        if great_circle(origin, lsoa_coordinates).km <= radius / 1000:
            lsoas_in_radius.append(lsoa_code)


    # calculate shortest paths from origin to LSOAs within search radius
    travel_times = []
    for lsoa_code in lsoas_in_radius:
        destination = (lsoa_population[lsoa_code]['Latitude'], lsoa_population[lsoa_code]['Longitude'])
        try:
            route = nx.shortest_path(G, source=ox.distance.nearest_nodes(G, origin[1], origin[0]),
                                     target=ox.distance.nearest_nodes(G, destination[1], destination[0]), 
                                     weight='travel_time')
            travel_time = sum([G[u][v][0]['travel_time'] for u, v in zip(route[:-1], route[1:])])
            travel_times.append(travel_time)
        except nx.NetworkXNoPath:
            pass

    # calculate average travel time and population within search radius
    avg_travel_time = sum(travel_times) / len(travel_times) / 60
    population_covered = sum([lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius])

    return avg_travel_time, population_covered


In [ ]:
avg_travel_time, pop_covered = get_average_travel_times('184 Victoria Road, Cambridge, UK', 5000, lsoa_population, G)
print(f"Average travel time: {avg_travel_time} minutes")
print(f"Population covered: {pop_covered}")


In [ ]:
total_population = sum([data['Population'] for lsoa, data in lsoa_population.items()])
total_population

In [ ]:
radius

In [ ]:
len(lsoas_in_radius)

In [ ]:
import folium

location = ox.geocode('184 Victoria Road, Cambridge, UK')

m = folium.Map(location=location, zoom_start=12)
folium.Marker(location=location, tooltip='Address').add_to(m)
folium.Circle(location=location, radius=10000, color='red', fill=False, tooltip='Search Radius').add_to(m)

for lsoa_code in lsoas_in_radius:
    row = gdf_cambs.loc[gdf_cambs['LSOA21CD'] == lsoa_code].iloc[0]
    
    if row['geometry'].geom_type == 'Polygon':
        lsoa_boundary = [tuple(reversed(coord)) for coord in list(row['geometry'].exterior.coords)]
    elif row['geometry'].geom_type == 'MultiPolygon':
        largest_polygon = max(row['geometry'], key=lambda x: x.area) 
        lsoa_boundary = [tuple(reversed(coord)) for coord in list(largest_polygon.exterior.coords)]


    lsoa_polygon = folium.Polygon(
        locations=lsoa_boundary,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
        tooltip=lsoa_code,
    )

   
    lsoa_polygon.add_to(m)


m

In [ ]:
print(f"Average travel time: {avg_travel_time} minutes")
print(f"Population covered: {pop_covered}")

In [ ]:
import folium
location = ox.geocode('184 Victoria Road, Cambridge, UK')
m = folium.Map(location=location, zoom_start=12)
folium.Marker(location=location, tooltip='Address').add_to(m)
folium.Circle(location=location, radius=10000, color='red', fill=False, tooltip='Search Radius').add_to(m)


In [ ]:

for lsoa_code in lsoas_in_radius:
    lsoa_data = lsoa_population[lsoa_code]
    popup_text = f"LSOA code: {lsoa_code}<br>Population: {lsoa_data['Population']}"
    folium.Marker(location=[lsoa_data['Latitude'], lsoa_data['Longitude']], 
                  popup=popup_text, 
                  tooltip=lsoa_data['Population']).add_to(m)

# add LSOA boundaries to map
folium.GeoJson(gdf_cambs[gdf_cambs['LSOA21CD'].isin(lsoas_in_radius)]).add_to(m)


In [ ]:
m

In [ ]:
  #geocode address using Nominatim
geolocator = Nominatim(user_agent='my_application')
location = Nominatim(user_agent='my_app').geocode(address)
origin = (location.latitude, location.longitude)
# get LSOAs within search radius


In [ ]:
lsoas_in_radius = []
for lsoa_code, data in lsoa_population.items():
    if distance(origin, (data['Latitude'], data['Longitude'])).km <= radius/1000:
        lsoas_in_radius.append(lsoa_code)
# calculate shortest paths from origin to LSOAs within search radius


In [ ]:
len(lsoa_population)

In [ ]:
distance(origin, (data['Latitude'], data['Longitude']))

In [ ]:
lsoas_in_radius

In [ ]:
travel_times = []
for lsoa_code in lsoas_in_radius:
    destination = (lsoa_population[lsoa_code]['Latitude'], lsoa_population[lsoa_code]['Longitude'])
    try:
        route = nx.shortest_path(G, source=ox.distance.nearest_nodes(G, origin[1], origin[0]),
                                 target=ox.distance.nearest_nodes(G, destination[1], destination[0]), 
                                 weight='travel_time')
        travel_time = sum([G[u][v][0]['travel_time'] for u, v in zip(route[:-1], route[1:])])
        travel_times.append(travel_time)
    except nx.NetworkXNoPath:
        pass


In [ ]:
# calculate average travel time and population within search radius
avg_travel_time = sum(travel_times) / len(travel_times) / 60
population_covered = sum([lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius])

In [ ]:
lsoas_and_populations = {lsoa: lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius}


In [ ]:
lsoas_and_populations

In [ ]:
population_covered = sum([lsoa_population[lsoa]['Population'] for lsoa in lsoas_in_radius])

